# Классификация тональности коротких текстов [SpaCy]

In [6]:
import spacy
import pandas as pd
from tqdm.auto import tqdm
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split


### Работа с датасетом

In [8]:
# Загрузить датасет
df = pd.read_csv('data/women_clothing.csv', encoding='utf8', sep='\t')
df


,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative
...,...,...
89995,сделано достаточно хорошо. на ткани сделан рис...,positive
89996,Накидка шикарная. Спасибо большое провдо линяе...,positive
89997,спасибо большое ) продовца рекомендую.. заказа...,positive
89998,Очень довольна заказом! Меньше месяца в РБ. К...,positive


In [9]:
# убрать класс neutral
df = df[df['sentiment'] != 'neautral']


In [10]:
# Разделить данные на обучающий и валидационный наборы
train_data, valid_data = train_test_split(df, test_size=0.3, random_state=42)


### list(tuple(text, label)) -> List(spacy.Doc.doc)

In [11]:
! python -m spacy download ru_core_news_sm

     ---------------------------------------- 0.0/15.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/15.3 MB 1.4 MB/s eta 0:00:12
     --------------------------------------- 0.1/15.3 MB 656.4 kB/s eta 0:00:24
     --------------------------------------- 0.1/15.3 MB 939.4 kB/s eta 0:00:17
      --------------------------------------- 0.2/15.3 MB 1.3 MB/s eta 0:00:12
      --------------------------------------- 0.3/15.3 MB 1.5 MB/s eta 0:00:10
     - -------------------------------------- 0.5/15.3 MB 2.0 MB/s eta 0:00:08
     -- ------------------------------------- 0.9/15.3 MB 3.0 MB/s eta 0:00:05
     --- ------------------------------------ 1.2/15.3 MB 3.3 MB/s eta 0:00:05
     --- ------------------------------------ 1.3/15.3 MB 3.5 MB/s eta 0:00:04
     --- ------------------------------------ 1.3/15.3 MB 3.5 MB/s eta 0:00:04
     --- ------------------------------------ 1.4/15.3 MB 3.0 MB/s eta 0:00:05
     --- ------------------------------------ 1.4/15.3 MB

In [12]:
nlp = spacy.load("ru_core_news_sm") # небольшой на русском

In [13]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        # One Hot Encodding
        if label == 'negative':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

In [14]:
train_docs = make_docs(train_data.values)

  0%|          | 0/42000 [00:00<?, ?it/s]

In [15]:
valid_docs = make_docs(valid_data.values)

  0%|          | 0/18000 [00:00<?, ?it/s]

In [16]:
# Запись на диск
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")

doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")


### Работа с SpaCy

https://spacy.io/usage/training#quickstart

base_config:

[paths]<br />
train = train.spacy<br />
dev = valid.spacy<br />
vectors = null<br />
[system]<br />
gpu_allocator = null<br />

[nlp]<br />
lang = "ru"<br />
pipeline = ["textcat"]<br />
batch_size = 1000<br />

[components]<br />

[components.textcat]<br />
factory = "textcat"<br />

In [17]:
# на основе base_config сделать config.cfg
! python -m spacy init fill-config base_config.cfg config.cfg


Usage: python -m spacy init fill-config [OPTIONS] BASE_PATH [OUTPUT_FILE]
Try 'python -m spacy init fill-config --help' for help.

Error: Invalid value for 'BASE_PATH': File 'base_config.cfg' does not exist.


config:

[training]<br />
dev_corpus = "corpora.dev"<br />
train_corpus = "corpora.train"<br />
seed = ${system.seed}<br />
gpu_allocator = \${system.gpu_allocator}<br />
dropout = 0.1<br />
accumulate_gradient = 1<br />
patience = 1600<br />
max_epochs = 2<br />
max_steps = 4000

In [20]:
# тренируем модель
! python -m spacy train config.cfg --output ./output


✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\nikgo\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\__main__.py", line 4, in <module>
    setup_cli()
  File "c:\Users\nikgo\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\cli\_util.py", line 92, in setup_cli
    command(prog_name=COMMAND)
  File "c:\Users\nikgo\AppData\Local\Programs\Python\Python311\Lib\site-packages\click\core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nikgo\AppData\Local\Programs\Python\Python311\Lib\site-packages\typer\core.py", line 778, in main
    return _main(
           ^^^^^^
  File "c:\Users\nikgo\AppData\Local\Programs\Python\Python311\Lib\site-packages\typer\core.py", line 216, in _main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "c:\Users\nikgo\AppData\Local\Programs\Python\

### Тест модели

In [22]:
# загружаем лучшую модель
nlp = spacy.load("output/model-best")
texts = ["Заказывала фиолетовую кофту размер XL, пришла очень маленькая кофта по длине 53 см и по груди тоже-детская разм. меньше чем S. Очень расстроилась, по спору присудили возврат денег 27 декабря. Выплата от 3 до 20 дней. Прошло уже 24 дня, деньги продавец не вернул, на сообщения не отвечает! Кофту такого смешного размера носить нельзя. Продавец непорядочный! Непонятно, почему Алиэкспресс после спора не отслеживают возвраты денег!",
         "Ужасная синтетика! Тонкая, ничего общего с представленной картинкой, не яркая, рисунок растянут и тусклый, впрочем как и сама кофта- мешок! На картинке кажется приталенной на самом деле нет! Не рекомендую",
         "Немного маловато,но жене понравилось",
         "Куртка без дефектов, без запаха, доставка месяц. Материал тонкий , просвечивает наполнитель.  На ог 108 ,  от. 86 размер xxx сел отлично."]

print("type : ‘quit’ to exit")
# predict the sentiment until someone writes quit
for text in texts:
    print(text)
    doc = nlp(text)
    print(doc.cats)
    if doc.cats['positive'] > .5:
        print(f"the sentiment is positive")
    else:
        print(f"the sentiment is negative")


OSError: [E050] Can't find model 'output/model-best'. It doesn't seem to be a Python package or a valid path to a data directory.